In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
import time
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

from libworm.torch.beta_net import BetaNeuronNet, from_connectome
from libworm.data import connectomes, traces
from libworm import preprocess
from libworm.functions import set_neurons, tcalc_s_inf, set_trace

In [6]:
trace, _, trace_labels, label2index, timestamps = traces.load_trace()
timestamps = timestamps - timestamps[0]
chemical, gapjn = connectomes.load_cook_connectome()
neurons = connectomes.get_main_neurons(chemical, gapjn)
neurons.sort(key=lambda item: f"AAA{label2index[item]:04d}{item}" if item in label2index else item)
default_model = from_connectome(chemical, gapjn, neurons) 
model = from_connectome(chemical, gapjn, neurons)
checkpoint = torch.load("checkpoints/ruby/modellr001dt01.pt")
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [10]:
for param in default_model.parameters():
    print(param)

Parameter containing:
tensor([10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10.,

In [11]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([10.8221, 10.3441, 11.1028, 11.3035, 10.8932, 10.8286, 11.1675, 11.1315,
        11.1511, 11.2967,  9.9940,  9.9940, 11.1430,  9.9940, 11.1452, 11.1210,
        10.0060, 10.0060, 10.2759, 10.8314, 10.0997, 11.2851, 10.5514, 10.2738,
        11.1676,  9.9940, 11.2412, 11.0795, 11.4046, 10.2729, 11.0858,  9.9940,
        11.0793,  9.9940, 11.1042, 11.1006, 10.0060, 11.1660,  9.9940, 11.1888,
        10.0060, 11.1370, 11.1936, 10.8033, 11.0787, 10.0060,  9.9940, 11.1657,
        11.1026, 10.3155, 11.0857, 11.2116, 11.3876, 10.8435, 11.1903, 11.0983,
        11.1908,  9.9940, 10.8257,  9.9940, 11.1315, 11.0846, 10.9134, 11.1393,
        10.0060, 10.0060, 11.2095, 10.5509, 11.1931, 11.1180, 11.1872, 10.8212,
        10.5516, 10.8306, 11.1883,  9.9940, 10.5805, 11.1703, 11.1770, 10.8571,
        10.8317, 10.0060,  9.9940, 10.3859, 10.5512, 10.3519, 10.2720, 10.0060,
        10.8317, 10.0060, 11.2422, 10.9786, 11.3654, 10.5445, 11.0785, 11.1454,
         9.9940, 1